In [1]:
suppress = True

In [2]:
import torch
from torch import nn
from qtorch.quant import *
from qtorch.auto_low import *

class SimpleLinearLP(nn.Module):
    def __init__(self, quant, size=3072):
        super(SimpleLinearLP, self).__init__()
        self.classifier = nn.Linear(size, size)
        self.classifier.weight.data = torch.ones_like(self.classifier.weight.data)
        self.classifier.bias.data = torch.ones_like(self.classifier.bias.data)
        self.quant = quant

    def forward(self, x):
        x = x.view(1, x.size(0))
        x = self.classifier(x)
        x = self.quant(x)
        return x
    
class SimpleLinearManualAutoLP(nn.Module):
    def __init__(self, quant, size=3072):
        super(SimpleLinearManualAutoLP, self).__init__()
        self.classifier = nn.Linear(size, size)
        self.classifier.weight.data = torch.ones_like(self.classifier.weight.data)
        self.classifier.bias.data = torch.ones_like(self.classifier.bias.data)
        self.quant = quant
        old_forward = self.classifier.forward
        self.classifier.forward = lambda *inputs : self.quant(old_forward(*inputs))

    def forward(self, x):
        x = x.view(1, x.size(0))
        x = self.classifier(x)
#         x = self.quant(x)
        return x

class SimpleLinear(nn.Module):
    def __init__(self, size=3072):
        super(SimpleLinear, self).__init__()
        self.classifier = nn.Linear(size, size)
        self.classifier.weight.data = torch.ones_like(self.classifier.weight.data)
        self.classifier.bias.data = torch.ones_like(self.classifier.bias.data)

    def forward(self, x):
        x = x.view(1, x.size(0))
        x = self.classifier(x)
        return x

    
class SimpleMatrix(nn.Module):
    def __init__(self, size=3072):
        super(SimpleMatrix, self).__init__()
        self.classifier = nn.Linear(size, size)
        self.classifier.weight.data = torch.ones_like(self.classifier.weight.data)
        self.classifier.bias.data = torch.ones_like(self.classifier.bias.data)

    def forward(self, x):
        x = self.classifier(x)
        return x

class SimpleMatrixLP(nn.Module):
    def __init__(self, quant, size=3072):
        super(SimpleMatrixLP, self).__init__()
        self.classifier = nn.Linear(size, size)
        self.classifier.weight.data = torch.ones_like(self.classifier.weight.data)
        self.classifier.bias.data = torch.ones_like(self.classifier.bias.data)
        self.quant = quant

    def forward(self, x):
        x = self.classifier(x)
        x = self.quant(x)
        return x

class SimpleConv(nn.Module):
    def __init__(self, num_channels=10, kernel_size=3):
        super(SimpleConv, self).__init__()
        self.conv1 = nn.Conv2d(3, num_channels, kernel_size, padding=1)
        self.conv1.weight.data = torch.ones_like(self.conv1.weight.data)
        self.conv1.bias.data = torch.ones_like(self.conv1.bias.data)

        self.classifier = nn.Conv2d(num_channels, num_channels, kernel_size, padding=1)
        self.classifier.weight.data = torch.ones_like(self.classifier.weight.data)
        self.classifier.bias.data = torch.ones_like(self.classifier.bias.data)

    def forward(self, x):
        x = self.conv1(x)
        x = self.classifier(x)
        return x

class SimpleConvLP(nn.Module):
    def __init__(self, quant, num_channels=10, kernel_size=3):
        super(SimpleConvLP, self).__init__()
        self.conv1 = nn.Conv2d(3, num_channels, kernel_size, padding=1)
        self.conv1.weight.data = torch.ones_like(self.conv1.weight.data)
        self.conv1.bias.data = torch.ones_like(self.conv1.bias.data)
        self.quant = quant
        self.classifier = nn.Conv2d(num_channels, num_channels, kernel_size, padding=1)
        self.classifier.weight.data = torch.ones_like(self.classifier.weight.data)
        self.classifier.bias.data = torch.ones_like(self.classifier.bias.data)

    def forward(self, x):
        x = self.conv1(x)
        x = self.quant(x)
        x = self.classifier(x)
        x = self.quant(x)
        return x




In [3]:
suppress = True # Suppress printing statements

import contextlib
import sys

# class DummyFile(object):
#     def write(self, x): pass
#     def flush(self): pass
import io

import functools
def suppress_print(func):
    if not suppress:
        return func
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        save_stdout = sys.stdout
        if suppress:
            sys.stdout = io.StringIO()
        value = func(*args, **kwargs)
        sys.stdout = save_stdout
        return value
    return wrapper_decorator

In [ ]:
# linear_timing()
quant = Quantizer(16, 16, 16, 16, -1, -1, -1, -1, "nearest", "nearest", "fixed", "fixed")
device="cpu"
model_1 = SimpleLinearLP(quant, size=10).to(device)
a = torch.linspace(- 2 ** (16-16-1), 2 ** (16-16-1) - 2 ** (-16), steps=10, device=device)
output_1 = model_1(a)

In [ ]:
@suppress_print
def test():
    print("nmb")
test()

In [ ]:
@suppress_print
def conv_timing(conv_channel=512, input_size=3072, wl=16, fl=16, man=8, exp=7, rounding_type="nearest", number_type="fixed", device="cuda"):
    
    if number_type in ["fixed", "block"]:
        quant = Quantizer(wl, wl, fl, fl, -1, -1, -1, -1, rounding_type, rounding_type, number_type, number_type)
    else:
        quant = Quantizer(-1, -1, -1, -1, man, man, exp, exp, rounding_type, rounding_type, number_type, number_type)
    
    import math
    width = int(math.sqrt(input_size / 3))
    a = torch.linspace(- 2 ** (wl-fl-1), 2 ** (wl-fl-1) - 2 ** (-fl), steps=input_size, device=device)
    a = a.reshape(1,3, width,width)
    a_copy = a.clone()
    print(f"Test convolutional layer with {conv_channel} channels")
    model_1 = SimpleConvLP(quant, num_channels=conv_channel).to(device)
    model_2 = SimpleConv(num_channels=conv_channel).to(device)
    model_3 = SimpleConv(num_channels=conv_channel).to(device)
    lower(model_2, 
          layer_types=["conv"],
          wl_activate=wl, 
          wl_error=wl,
          fl_activate=fl, 
          fl_error=fl,
          activate_rounding=rounding_type,
          error_rounding=rounding_type,
          activate_type=number_type,
          error_type=number_type
    )
    
    import time
    output_1 = model_1(a)
    output_1 = model_1(a)
    output_1 = model_1(a)
    loop = 1000
    total_time = 0
    for i in range(loop):
        time_now = time.time()
        output_1 = model_1(a)
        time_since_1 = time.time() - time_now
        total_time += time_since_1

    output_2 = model_2(a_copy)
    output_2 = model_2(a_copy)
    output_2 = model_2(a_copy)
    total_time_2 = 0
    for i in range(loop):
        time_now = time.time()
        output_2 = model_2(a_copy)
        time_since_2 = time.time() - time_now
        total_time_2 += time_since_2

    output_3 = model_3(a_copy)
    output_3 = model_3(a_copy)
    output_3 = model_3(a_copy)
    total_time_3 = 0
    for i in range(loop):
        time_now = time.time()
        output_3 = model_3(a_copy)
        time_since_3 = time.time() - time_now
        total_time_3 += time_since_3

    # print(torch.nn.L1Loss()(a,a_copy))
    # print(torch.nn.L1Loss()(output_1,output_2))
    # print(torch.nn.L1Loss()(output_1,output_3))
    print(f"Running time - Model with manually inserted LP layers: {total_time}")
    print(f"Running time - Model with auto inserted LP layers: {total_time_2}")
    print(f"Running time - Model with no LP layers: {total_time_3}")
    return total_time, total_time_2, total_time_3

In [ ]:
@suppress_print
def linear_timing(input_size=10, wl=16, fl=16, rounding_type="nearest", number_type="fixed", device="cuda"):
    import pdb; pdb.set_trace();
    if number_type in ["fixed", "block"]:
        quant = Quantizer(wl, wl, fl, fl, -1, -1, -1, -1, rounding_type, rounding_type, number_type, number_type)
    else:
        quant = Quantizer(-1, -1, -1, -1, man, man, exp, exp, rounding_type, rounding_type, number_type, number_type)
    
    matrix_size = input_size
    a = torch.linspace(- 2 ** (wl-fl-1), 2 ** (wl-fl-1) - 2 ** (-fl), steps=matrix_size, device=device)
    a_copy = a.clone()
    print(f"Test linear layer with size {matrix_size} x {matrix_size}")
    model_1 = SimpleLinearLP(quant, size=matrix_size).to(device)
    model_2 = SimpleLinear(size=matrix_size).to(device)
    model_3 = SimpleLinear(size=matrix_size).to(device)
    model_4 = SimpleLinearManualAutoLP(quant, size=matrix_size).to(device)
    lower(model_2, 
          layer_types=["linear"],
          wl_activate=wl, 
          wl_error=wl,
          fl_activate=fl, 
          fl_error=fl,
          activate_rounding=rounding_type,
          error_rounding=rounding_type,
          activate_type=number_type,
          error_type=number_type
    )
    import time
    output_1 = model_1(a)
    output_1 = model_1(a)
    output_1 = model_1(a)
    loop = 1000
    total_time = 0
    for i in range(loop):
        time_now = time.time()
        output_1 = model_1(a)
        time_since_1 = time.time() - time_now
        total_time += time_since_1

    output_2 = model_2(a_copy)
    output_2 = model_2(a_copy)
    output_2 = model_2(a_copy)
    total_time_2 = 0
    for i in range(loop):
        time_now = time.time()
        output_2 = model_2(a_copy)
        time_since_2 = time.time() - time_now
        total_time_2 += time_since_2

    output_3 = model_3(a_copy)
    output_3 = model_3(a_copy)
    output_3 = model_3(a_copy)
    total_time_3 = 0
    for i in range(loop):
        time_now = time.time()
        output_3 = model_3(a_copy)
        time_since_3 = time.time() - time_now
        total_time_3 += time_since_3
    
    output_4 = model_4(a_copy)
    output_4 = model_4(a_copy)
    output_4 = model_4(a_copy)
    total_time_4 = 0
    for i in range(loop):
        time_now = time.time()
        output_4 = model_4(a_copy)
        time_since_4 = time.time() - time_now
        total_time_4 += time_since_4

    # print(torch.nn.L1Loss()(a,a_copy))
    # print(torch.nn.L1Loss()(output_1,output_2))
    # print(torch.nn.L1Loss()(output_1,output_3))
    print(f"Running time - Model with manually inserted LP layers: {total_time}")
    print(f"Running time - Model with auto inserted LP layers: {total_time_2}")
    print(f"Running time - Model with no LP layers: {total_time_3}")
    print(f"Running time - Model with manually inserted LP (lambda) layers: {total_time_4}")
    return total_time, total_time_2, total_time_3

In [ ]:
@suppress_print
def matrix_timing(input_size=3072, wl=16, fl=16, rounding_type="nearest", number_type="fixed", device="cuda"):

    if number_type in ["fixed", "block"]:
        quant = Quantizer(wl, wl, fl, fl, -1, -1, -1, -1, rounding_type, rounding_type, number_type, number_type)
    else:
        quant = Quantizer(-1, -1, -1, -1, man, man, exp, exp, rounding_type, rounding_type, number_type, number_type)
    
    matrix_size = input_size
    a = torch.linspace(- 2 ** (wl-fl-1), 2 ** (wl-fl-1) - 2 ** (-fl), steps=matrix_size, device=device)
    a = a.repeat(a.size(0))
    a = a.reshape(matrix_size, matrix_size)
    a_copy = a.clone()
    print(f"Test matrix x matrix with size {matrix_size} x {matrix_size}")
    model_1 = SimpleMatrixLP(quant, size=matrix_size).to(device)
    model_2 = SimpleMatrix(size=matrix_size).to(device)
    model_3 = SimpleMatrix(size=matrix_size).to(device)
    lower(model_2, 
          layer_types=["linear"],
          wl_activate=wl, 
          wl_error=wl,
          fl_activate=fl, 
          fl_error=fl,
          activate_rounding=rounding_type,
          error_rounding=rounding_type,
          activate_type=number_type,
          error_type=number_type
    )
    import time
    output_1 = model_1(a)
    output_1 = model_1(a)
    output_1 = model_1(a)
    loop = 1000
    total_time = 0
    for i in range(loop):
        time_now = time.time()
        output_1 = model_1(a)
        time_since_1 = time.time() - time_now
        total_time += time_since_1

    output_2 = model_2(a_copy)
    output_2 = model_2(a_copy)
    output_2 = model_2(a_copy)
    total_time_2 = 0
    for i in range(loop):
        time_now = time.time()
        output_2 = model_2(a_copy)
        time_since_2 = time.time() - time_now
        total_time_2 += time_since_2

    output_3 = model_3(a_copy)
    output_3 = model_3(a_copy)
    output_3 = model_3(a_copy)
    total_time_3 = 0
    for i in range(loop):
        time_now = time.time()
        output_3 = model_3(a_copy)
        time_since_3 = time.time() - time_now
        total_time_3 += time_since_3

    # print(torch.nn.L1Loss()(a,a_copy))
    # print(torch.nn.L1Loss()(output_1,output_2))
    # print(torch.nn.L1Loss()(output_1,output_3))
    print(f"Running time - Model with manually inserted LP layers: {total_time}")
    print(f"Running time - Model with auto inserted LP layers: {total_time_2}")
    print(f"Running time - Model with no LP layers: {total_time_3}")
    return total_time, total_time_2, total_time_3

In [ ]:
def timeing_plot(func, input_sizes, wl=16, fl=16, rounding_type="nearest", number_type="fixed", device="cpu", title="Running Time of Linear layer", xlabel="Input Size"):

    time_1_arr = []
    time_2_arr = []
    time_3_arr = []
    for input_size in input_sizes:
        time_1, time_2, time_3 = func(input_size, wl=wl, fl=fl, rounding_type=rounding_type, number_type=number_type, device=device)
        time_1_arr.append(time_1)
        time_2_arr.append(time_2)
        time_3_arr.append(time_3)
    import numpy as np
    import matplotlib.pyplot as plt
    %matplotlib inline
#     input_sizes = [500, 1000, 1500, 2000, 2500, 3000, 5000, 8000]
    # evenly sampled time at 200ms intervals
    time_1_np = np.array(time_1_arr)
    time_2_np = np.array(time_2_arr)
    time_3_np = np.array(time_3_arr)
    # red dashes, blue squares and green triangles
    plt.xlabel(xlabel)
    plt.ylabel('Running Time')
    plt.title(title)
    plt.plot(input_sizes, time_1_np, 'r--', label= "Manually Inserted LP")
    plt.plot(input_sizes, time_2_np, 'bs--', label="Auto Inserted LP")
    plt.plot(input_sizes, time_3_np, 'g--',  label="No LP")
    plt.legend(loc='upper left',bbox_to_anchor=(1,1))
    plt.show()

In [ ]:
input_sizes = [500, 1000, 1500, 2000, 2500, 3000, 3500]
input_sizes = [5]

In [ ]:
timeing_plot(linear_timing, 
             input_sizes, 
             rounding_type="nearest", 
             number_type="fixed", 
             device="cpu", 
             title="Running Time of Linear layer (cpu + nearest_round + fixed point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(linear_timing, 
             input_sizes, 
             rounding_type="stochastic", 
             number_type="fixed", 
             device="cpu", 
             title="Running Time of Linear layer (cpu + stochastic_round + fixed point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(linear_timing, 
             input_sizes, 
             rounding_type="nearest", 
             number_type="block", 
             device="cpu", 
             title="Running Time of Linear layer (cpu + nearest_round + block floating point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(linear_timing, 
             input_sizes, 
             rounding_type="stochastic", 
             number_type="block", 
             device="cpu", 
             title="Running Time of Linear layer (cpu + stochastic_round + block floating point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(linear_timing, 
             input_sizes, 
             rounding_type="nearest", 
             number_type="float", 
             device="cpu", 
             title="Running Time of Linear layer (cpu + nearest_round + floating point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(linear_timing, 
             input_sizes, 
             rounding_type="stochastic", 
             number_type="float", 
             device="cpu", 
             title="Running Time of Linear layer (cpu + stochastic_round + floating point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(linear_timing, 
             input_sizes, 
             rounding_type="nearest", 
             number_type="fixed", 
             device="cuda", 
             title="Running Time of Linear layer (cuda + nearest_round + fixed point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(linear_timing, 
             input_sizes, 
             rounding_type="stochastic", 
             number_type="fixed", 
             device="cuda", 
             title="Running Time of Linear layer (cuda + stochastic_round + fixed point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(linear_timing, 
             input_sizes, 
             rounding_type="nearest", 
             number_type="block", 
             device="cuda", 
             title="Running Time of Linear layer (cuda + nearest_round + block floating point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(linear_timing, 
             input_sizes, 
             rounding_type="stochastic", 
             number_type="block", 
             device="cuda", 
             title="Running Time of Linear layer (cuda + stochastic_round + block floating point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(linear_timing, 
             input_sizes, 
             rounding_type="nearest", 
             number_type="float", 
             device="cuda", 
             title="Running Time of Linear layer (cuda + nearest_round + floating point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(linear_timing, 
             input_sizes, 
             rounding_type="stochastic", 
             number_type="float", 
             device="cuda", 
             title="Running Time of Linear layer (cuda + stochastic_round + floating point)", 
             xlabel="Input Size")

In [ ]:
input_sizes = [500, 1000, 1500, 2000, 2500, 3000, 3500]

In [ ]:
timeing_plot(matrix_timing, 
             input_sizes, 
             rounding_type="nearest", 
             number_type="fixed", 
             device="cpu", 
             title="Running Time of Matrix x Matrix (cpu + nearest_round + fixed point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(matrix_timing, 
             input_sizes, 
             rounding_type="stochastic", 
             number_type="fixed", 
             device="cpu", 
             title="Running Time of Matrix x Matrix (cpu + stochastic_round + fixed point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(matrix_timing, 
             input_sizes, 
             rounding_type="nearest", 
             number_type="block", 
             device="cpu", 
             title="Running Time of Matrix x Matrix (cpu + nearest_round + block floating point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(matrix_timing, 
             input_sizes, 
             rounding_type="stochastic", 
             number_type="block", 
             device="cpu", 
             title="Running Time of Matrix x Matrix (cpu + stochastic_round + block floating point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(matrix_timing, 
             input_sizes, 
             rounding_type="nearest", 
             number_type="float", 
             device="cpu", 
             title="Running Time of Matrix x Matrix (cpu + nearest_round + floating point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(matrix_timing, 
             input_sizes, 
             rounding_type="stochastic", 
             number_type="float", 
             device="cpu", 
             title="Running Time of Matrix x Matrix (cpu + stochastic_round + floating point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(matrix_timing, 
             input_sizes, 
             rounding_type="nearest", 
             number_type="fixed", 
             device="cuda", 
             title="Running Time of Matrix x Matrix (cuda + nearest_round + fixed point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(matrix_timing, 
             input_sizes, 
             rounding_type="stochastic", 
             number_type="fixed", 
             device="cuda", 
             title="Running Time of Matrix x Matrix (cuda + stochastic_round + fixed point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(matrix_timing, 
             input_sizes, 
             rounding_type="nearest", 
             number_type="block", 
             device="cuda", 
             title="Running Time of Matrix x Matrix (cuda + nearest_round + block floating point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(matrix_timing, 
             input_sizes, 
             rounding_type="stochastic", 
             number_type="block", 
             device="cuda", 
             title="Running Time of Matrix x Matrix (cuda + stochastic_round + block floating point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(matrix_timing, 
             input_sizes, 
             rounding_type="nearest", 
             number_type="float", 
             device="cuda", 
             title="Running Time of Matrix x Matrix (cuda + nearest_round + floating point)", 
             xlabel="Input Size")

In [ ]:
timeing_plot(matrix_timing, 
             input_sizes, 
             rounding_type="stochastic", 
             number_type="float", 
             device="cuda", 
             title="Running Time of Matrix x Matrix (cuda + stochastic_round + floating point)", 
             xlabel="Input Size")

In [ ]:
channel_sizes = [500, 1000, 1500, 2000, 2500, 3000, 3500]
channel_sizes = [1]

In [ ]:
timeing_plot(conv_timing, 
             channel_sizes, 
             rounding_type="nearest", 
             number_type="fixed", 
             device="cpu", 
             title="Running Time of Convolutional layer  (cpu + nearest_round + fixed point)", 
             xlabel="Channel Number")

In [ ]:
timeing_plot(conv_timing, 
             channel_sizes, 
             rounding_type="stochastic", 
             number_type="fixed", 
             device="cpu", 
             title="Running Time of Convolutional layer (cpu + stochastic_round + fixed point)", 
             xlabel="Channel Number")

In [ ]:
timeing_plot(conv_timing, 
             channel_sizes, 
             rounding_type="nearest", 
             number_type="block", 
             device="cpu", 
             title="Running Time of Convolutional layer (cpu + nearest_round + block floating point)", 
             xlabel="Channel Number")

In [ ]:
timeing_plot(conv_timing, 
             channel_sizes, 
             rounding_type="stochastic", 
             number_type="block", 
             device="cpu", 
             title="Running Time of Convolutional layer (cpu + stochastic_round + block floating point)", 
             xlabel="Channel Number")

In [ ]:
timeing_plot(conv_timing, 
             channel_sizes, 
             rounding_type="nearest", 
             number_type="float", 
             device="cpu", 
             title="Running Time of Convolutional layer (cpu + nearest_round + floating point)", 
             xlabel="Channel Number")

In [ ]:
timeing_plot(conv_timing, 
             channel_sizes, 
             rounding_type="stochastic", 
             number_type="float", 
             device="cpu", 
             title="Running Time of Convolutional layer (cpu + stochastic_round + floating point)", 
             xlabel="Channel Number")

In [ ]:
timeing_plot(conv_timing, 
             channel_sizes, 
             rounding_type="nearest", 
             number_type="fixed", 
             device="cuda", 
             title="Running Time of Convolutional layer (cuda + nearest_round + fixed point)", 
             xlabel="Channel Number")

In [ ]:
timeing_plot(conv_timing, 
             channel_sizes, 
             rounding_type="stochastic", 
             number_type="fixed", 
             device="cuda", 
             title="Running Time of Convolutional layer (cuda + stochastic_round + fixed point)", 
             xlabel="Channel Number")

In [ ]:
timeing_plot(conv_timing, 
             channel_sizes, 
             rounding_type="nearest", 
             number_type="block", 
             device="cuda", 
             title="Running Time of Convolutional layer (cuda + nearest_round + block floating point)", 
             xlabel="Channel Number")

In [ ]:
timeing_plot(conv_timing, 
             channel_sizes, 
             rounding_type="stochastic", 
             number_type="block", 
             device="cuda", 
             title="Running Time of Convolutional layer (cuda + stochastic_round + block floating point)", 
             xlabel="Channel Number")

In [ ]:
timeing_plot(conv_timing, 
             channel_sizes, 
             rounding_type="nearest", 
             number_type="float", 
             device="cuda", 
             title="Running Time of Convolutional layer (cuda + nearest_round + floating point)", 
             xlabel="Channel Number")

In [ ]:
timeing_plot(conv_timing, 
             channel_sizes, 
             rounding_type="stochastic", 
             number_type="float", 
             device="cuda", 
             title="Running Time of Convolutional layer (cuda + stochastic_round + floating point)", 
             xlabel="Channel Number")